### Embedding Adaptors
Embedding Adaptors are an AI retrieval optimization technique provided by chroma. In this section we will be experimenting with using embedding adpators on chroma to optimize the retrieval performance
